# Prepare sample point neighborhood stats within each study region
### *use odense as an example in this notebook
Description: This script is for preparing all sample points indicators/stats.  
All the cities should run this script first to get the pre-prepared sample points before running the aggregation scripts (aggr.py)

**City-specific input data**  

| Input data | Geometry | Description |
| --- | --- | --- |
| aos_nodes_30m_line | point | Public open space pseudo entry points (points on boundary of park every 20m within 30m of road) |
| clean_intersections_12m |	point |	Clean intersections (not required; counts are associated with pop_ghs_2015) |
| dest_type	| NA (non-spatial) |	Summary of destinations and counts |
| destinations |	point	| OSM destinations retrieved using specified definitions (only require: supermarkets, convenience,  pt_any --- use dest_name_full to determine, will need to combine convenience destinations) |
| pop_ghs_2015	| polygon	| 250m hex grid, associated with area_sqkm (fixed), population estimate (2015), population per sq km, intersection count, intersections per sq km |
| urban_sample_points |	point |	Sample points in urban region (every 30m along pedestrian network) |
| urban_study_region | polygon | Urban study region (intersection of city boundary and GHS 2015 urban centre layer) |


**Two outputs:**
1. average poplulation and intersection density per sample point
2. accessibility, daily living and walkability score per sample point

notice: must close the geopackage connection in QGIS.Otherwise, an error occurred when reading

In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import numpy as np
import os
import setup_sp as ssp # functions for setting up sample point stats used in this notebook
import setup_config as sc # import project config parameters

import time
from tqdm import tqdm
from multiprocessing import Pool, cpu_count, Value, Manager, Process
from functools import partial
import json
import fiona
import sys
import matplotlib.pyplot as plt
import numpy

%matplotlib inline

In [2]:
# output the processing city name to users
today = time.strftime("%Y-%m-%d")

# get the work directory
dirname = os.path.abspath("")

city = sys.argv[1]
configuration_file = f'configuration/vic.py'

assumptions = """
This code assumes the name of a known city to be passed as an argument, however none was provided.

Configuration python files containing the dictionaries 'config' and 'parameters' are written
to the ./configuration directory for cities through use of the set up configuration script setup_config.py,
like so: 
python setup_config.py auckland

or, to generate set up scripts for all cities
python setup_config.py
"""
    
try:
    exec(open(configuration_file).read())
except Exception as e:
    print(f"Failed to read configuration file {configuration_file}.\n\n{assumptions}")
    print(e)


In [3]:
print(f"\nGlobal indicators project {today}\n\nProcess city: {config['study_region'].title()}\n")

# geopackage path where to read all the required layers
gpkgPath = os.path.join(dirname, config["folder"], config["geopackagePath"])   

# define original graphml filepath
ori_graphml_filepath = os.path.join(dirname, config["folder"], config["graphmlName"])

if not os.path.exists(gpkgPath):
    # check if these files are located in the study region folder (ie. output location for pre-processing)
    alt_dir = f"./data/study_region/{config['study_region_full']}"
    alt_sources = (f"{alt_dir}/{os.path.basename(gpkgPath)}",
                   f"{alt_dir}/{os.path.basename(ori_graphml_filepath)}")
    if sum([os.path.exists(x) for x in alt_sources])==2:
        gpkgPath,ori_graphml_filepath = alt_sources
    else:
        sys.exit(f"\nThe required input files ({os.path.basename(gpkgPath)} and {os.path.basename(gpkgPath)}) "
         f"do not appear to exist in either the ./data/input folder or {alt_dir} folder.  "
         "Please ensure both of these file exist in one of these locations, or that the input "
         "configuration is correctly re-parameterised to recognise an alternative location.")


Global indicators project 2021-02-23

Process city: Vic



In [4]:
# geopackage path where to save processing layers
gpkgPath_output = os.path.join(dirname, config["folder"], config["geopackagePath_output"])

# Check if geopackage has a -wal file associated with it
# if so it is likely open and locked for use by another software package (e.g. QGIS)
# and will be unable to be used
for required_gpkg in [gpkgPath,gpkgPath_output]:
    if os.path.exists(f'{required_gpkg}-wal'):
        sys.exit(
        f"\nIt appears that the required geopackage {required_gpkg} may be open in another software package, " 
        "due to the presence of a Write Ahead Logging (WAL) file associated with it.  Please ensure that the input "  
        "geopackage is not being used in any other software before continuing, and that the file "
       f"'{required_gpkg}-wal' is not present before continuing."
       )

# read projected graphml filepath
proj_graphml_filepath = os.path.join(dirname, config["folder"], config["graphmlProj_name"])

G_proj = ssp.read_proj_graphml(proj_graphml_filepath,
                               ori_graphml_filepath, 
                               config["to_crs"],
                               undirected=True,
                               retain_fields=['osmid','length'])
    

Read network from disk.
  - Ensure graph is undirected.


In [5]:
# copy input geopackage to output geopackage, if not already exist
input_layers = fiona.listlayers(gpkgPath)
if not os.path.isfile(gpkgPath_output):
    print("Initialise sample point output geopackage as a copy of input geopackage")
    os.system(f'cp {gpkgPath} {gpkgPath_output}')
    output_layers = input_layers
else:
    output_layers = fiona.listlayers(gpkgPath_output)
    print("Sample point geopackage exists.")
    for layer in [x for x in input_layers if x not in output_layers]:
        print(f" - updating output geopackage to contain the layer '{layer}'")
        gpkgPath_input = gpd.read_file(gpkgPath, layer=layer)
        gpkgPath_input.to_file(gpkgPath_output, layer=layer, driver="GPKG")

# read hexagon layer of the city from disk, the hexagon layer is 250m*250m
# it should contain population estimates and intersection information
hexes = gpd.read_file(gpkgPath_output, layer=parameters["hex250"])
hexes.set_index('index',inplace=True)


Sample point geopackage exists.


## 1. Calculate average poplulation and intersection density for each sample point in study regions
**The steps are as follows:**
1. use the OSM pedestrain network (graphml in disk) to generate local 1600m neighborhood per urban sample points (sample point in disk)
2. load 250m hex grid from disk with population and network intersections density data
3. then calculate population and intersection density within each local walkable neighborhood (1600m) by averaging the hex level pop and intersection density data; final result is urban sample point dataframe with osmid, pop density, and intersection density.

In [6]:
print("\nFirst pass node-level neighbourhood analysis (Calculate average population and intersection density "
      "for each intersection node in study regions, taking mean values from distinct hexes within "
      "neighbourhood buffer distance)")
nh_startTime = time.time()
population_density = parameters["population_density"]
intersection_density = parameters["intersection_density"]
nh_fields_points = [population_density,intersection_density]
# read from disk if exist
if 'nodes_pop_intersect_density' in output_layers:                        
    print("  - Read population and intersection density from local file.")
    gdf_nodes_simple = gpd.read_file(gpkgPath_output, layer='nodes_pop_intersect_density')
    gdf_nodes_simple.set_index('osmid', inplace=True)
else:
    print("  - Set up simple nodes")
    gdf_nodes = ox.graph_to_gdfs(G_proj, nodes=True, edges=False)
    gdf_nodes.osmid = gdf_nodes.osmid.astype(int)
    gdf_nodes = gdf_nodes.drop_duplicates(subset="osmid")
    gdf_nodes.set_index('osmid', inplace=True)
    # associate nodes with hex_id
    gdf_nodes = ssp.spatial_join_index_to_gdf(gdf_nodes, hexes, right_index_name='hex_id',join_type='within')
    # keep only the unique node id column
    gdf_nodes = gdf_nodes[["hex_id","geometry"]]
    # drop any nodes which are na (they are outside the buffered study region and not of interest)
    gdf_nodes_simple = gdf_nodes[~gdf_nodes.hex_id.isna()].copy()
    gdf_nodes = gdf_nodes[["hex_id"]]

if len([x for x in nh_fields_points if x not in gdf_nodes_simple.columns]) > 0:
    # Calculate average population and intersection density for each intersection node in study regions
    # taking mean values from distinct hexes within neighbourhood buffer distance
    nh_fields_hex = ['pop_per_sqkm','intersections_per_sqkm']   
    # Create a dictionary of edge index and integer values of length
    # The length attribute was saved as string, so must be recast to use as weight
    # The units are meters, so the decimal precision is unnecessary (error is larger than this; meter is adequate)
    weight = dict(zip([k for k in G_proj.edges],[int(float(G_proj.edges[k]['length'])) for k in G_proj.edges]))

    # Add a new edge attribute using the integer weights
    nx.set_edge_attributes(G_proj, weight, 'weight')

    # run all pairs analysis
    total_nodes = len(gdf_nodes_simple)
    nh_distance = parameters["neighbourhood_distance"]
    print(f'  - Generate {nh_distance}m  neighbourhoods for nodes (All pairs Dijkstra shortest path analysis)')
    all_pairs_d = pd.DataFrame([(k,v.keys()) for k,v in tqdm(nx.all_pairs_dijkstra_path_length(G_proj,1000,'weight'),
                                        total=total_nodes,unit='nodes',desc=' '*18)],
                  columns = ['osmid','nodes']).set_index('osmid')
    # extract results
    print('  - Summarise attributes (average value from unique associated hexes within nh buffer distance)...')

    result = pd.DataFrame([tuple(hexes.loc[gdf_nodes.loc[all_pairs_d.loc[n].nodes,'hex_id'].dropna().unique(),    
                                    nh_fields_hex].mean().values) for index,n in    
                                        tqdm(np.ndenumerate(gdf_nodes_simple.index.values),total=total_nodes,desc=' '*18)],
                     columns = nh_fields_points,
                     index=gdf_nodes_simple.index.values)
    gdf_nodes_simple = gdf_nodes_simple.join(result)

    # save in geopackage (so output files are all kept together)
    gdf_nodes_simple.to_file(gpkgPath_output, layer='nodes_pop_intersect_density', driver="GPKG")



First pass node-level neighbourhood analysis (Calculate average population and intersection density for each intersection node in study regions, taking mean values from distinct hexes within neighbourhood buffer distance)
  - Read population and intersection density from local file.


In [7]:
# show sample point pop and intersection density data
gdf_nodes_simple.head()

,hex_id,sp_local_nh_avg_pop_density,sp_local_nh_avg_intersection_density,geometry
osmid,,,,
525877251,1651,5686.545508,129.574042,POINT (437779.504 4642789.855)
2838986757,237,28.767408,45.213903,POINT (436536.381 4637542.143)
1625751561,1784,1746.464938,89.680468,POINT (436586.031 4643103.392)
521781258,1055,1111.435046,39.459406,POINT (441362.048 4640785.311)
4377116692,2492,27.835967,32.060767,POINT (439640.738 4647011.830)


## Calculate accessibility to POI (supermarket,convenience,pt,pos), daily living and walkability for sample points
**steps as follow:**
   1. using pandana packadge to calculate distance to access from sample points to destinations (daily living destinations, public open space)
   2. calculate accessibiity score per sample point: transform accessibility distance to binary measure: 1 if access <= 500m, 0 otherwise
   3. calculate daily living score by summing the accessibiity scores to all POIs (excluding pos)
   4. calculate walkability score per sample point: get zscores for daily living accessibility, populaiton density and intersections pop_density; sum these three zscores at sample point level


In [8]:
print(f"Time taken to calculate or load city local neighbourhood statistics: {(time.time() - nh_startTime)/60:02g} mins")

# Calculate accessibility to POI (fresh_food_market,convenience,pt,pso) and
# walkability for sample points steps as follow:
# 1. using pandana packadge to calculate distance to access from sample
#    points to destinations (daily living destinations, public open space)
# 2. calculate accessibiity score per sample point: transform accessibility
#    distance to binary measure: 1 if access <= 500m, 0 otherwise
# 3. calculate daily living score by summing the accessibiity scores to all
#    POIs (excluding pos)
# 4. calculate walkability score per sample point: get zscores for daily
#    living accessibility, populaiton density and intersections population_density;
#    sum these three zscores at sample point level

print("\nCalculate assessbility to POIs.")
# read accessibility distance from configuration file, which is 500m

# create the pandana network, use network nodes and edges
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_proj)
network = ssp.create_pdna_net(gdf_nodes, gdf_edges, predistance=parameters["accessibility_distance"])

distance_results = {}
print("\nCalculating nearest node analyses ...")
for analysis_key in config['nearest_node_analyses']:
    print(f'\n\t- {analysis_key}')
    analysis = config['nearest_node_analyses'][analysis_key]
    layer_analysis_count = len(analysis['layers'])
    for layer in analysis['layers']:
        if layer is not None:
            output_names = analysis['output_names'].copy()
            if layer_analysis_count > 1 and layer_analysis_count==len(analysis['output_names']):
                # assume that output names correspond to layers, and refresh per analysis
                output_names = [output_names[analysis['layers'].index(layer)]]

            print(f'\t\t{output_names}')
            gdf_poi = gpd.read_file(f"data/{analysis['geopackage']}", layer = layer) 
            distance_results[f'{analysis}_{layer}'] = ssp.cal_dist_node_to_nearest_pois(gdf_poi, 
                                                         parameters["accessibility_distance"], 
                                                         network, 
                                                         category_field = analysis['category_field'],
                                                         categories = analysis['categories'],
                                                         filter_field = analysis['filter_field'],
                                                         filter_iterations = analysis['filter_iterations'],
                                                         output_names = output_names,
                                                         output_prefix = 'sp_nearest_node_')
        else:
            # create null results --- e.g. for GTFS analyses where no layer exists
            distance_results[f'{analysis_key}_{layer}'] = pd.DataFrame(index=gdf_nodes.index, 
                                    columns=[f'sp_nearest_node_{x}' for x in analysis['output_names']])

# concatenate analysis dataframes into one
gdf_nodes_poi_dist = pd.concat([gdf_nodes]+[distance_results[x] for x in distance_results], axis=1)

# set index of gdf_nodes_poi_dist, using 'osmid' as the index, and remove other unnecessary columns
gdf_nodes_poi_dist.set_index("osmid",inplace=True)
unnecessary_columns = [x for x in 
                         ["geometry", "id", "lat", "lon", "y", "x", "highway", "ref"] 
                            if x in gdf_nodes_poi_dist.columns]
gdf_nodes_poi_dist.drop(unnecessary_columns,axis=1, inplace=True, errors="ignore")

# replace -999 values (meaning no destination reached in less than 500 metres) as nan
gdf_nodes_poi_dist = round(gdf_nodes_poi_dist, 0).replace(-999, np.nan).astype("Int64")

Time taken to calculate or load city local neighbourhood statistics: 0.00890416 mins

Calculate assessbility to POIs.

Calculating nearest node analyses ...

	- Open street map destinations
		['fresh_food_market', 'convenience', 'pt_osm_any']

	- Public open space
		['public_open_space_any']
		['public_open_space_large']

	- Public transport (GTFS)


In [9]:
# read sample points from disk (in city-specific geopackage)
samplePointsData = gpd.read_file(gpkgPath_output, layer=parameters["samplePoints"])

# create 'hex_id' for sample point, if it not exists
if "hex_id" not in samplePointsData.columns:
    samplePointsData = ssp.spatial_join_index_to_gdf(samplePointsData, hexes, right_index_name='hex_id',join_type='within')


In [10]:
print("Restrict sample points to those not located in hexagons with a population below "
      f"the minimum threshold value ({parameters['pop_min_threshold']})..."),
below_minimum_pop_hex_ids = list(hexes.query(f'pop_est < {parameters["pop_min_threshold"]}').index.values)
sample_point_length_pre_discard = len(samplePointsData)
samplePointsData = samplePointsData[~samplePointsData.hex_id.isin(below_minimum_pop_hex_ids)]
sample_point_length_post_discard = len(samplePointsData)
print(f"  {sample_point_length_pre_discard - sample_point_length_post_discard} sample points discarded, "
      f"leaving {sample_point_length_post_discard} remaining.")

Restrict sample points to those not located in hexagons with a population below the minimum threshold value (5)...
  491 sample points discarded, leaving 11499 remaining.


In [11]:
print("Restrict sample points to those with two associated sample nodes..."),
sample_point_length_pre_discard = len(samplePointsData)
samplePointsData = samplePointsData.query(f"n1 in {list(gdf_nodes_simple.index.values)} "
                                          f"and n2 in {list(gdf_nodes_simple.index.values)}")
sample_point_length_post_discard = len(samplePointsData)
print(f"  {sample_point_length_pre_discard - sample_point_length_post_discard} sample points discarded, "
      f"leaving {sample_point_length_post_discard} remaining.")

samplePointsData.set_index("point_id", inplace=True)

distance_names = list(gdf_nodes_poi_dist.columns)

# Estimate full distance to destinations for sample points
full_nodes = ssp.create_full_nodes(
    samplePointsData,
    gdf_nodes_simple,
    gdf_nodes_poi_dist,
    distance_names,
    population_density,
    intersection_density,
)

samplePointsData = samplePointsData[["hex_id", "edge_ogc_fid", "geometry"]].join(full_nodes, how="left")

# create binary distances evaluated against accessibility distance
#binary_names = [f"{x.replace('nearest_node','access')}_binary" for x in distance_names]
#samplePointsData[binary_names] = (samplePointsData[distance_names] <= parameters['accessibility_distance']) \
 #                                      .astype("Int64").fillna(0)

Restrict sample points to those with two associated sample nodes...
  0 sample points discarded, leaving 11499 remaining.
Derive sample point estimates for accessibility and densities based on node distance relations
	 - match sample points whose locations coincide with intersections directly with intersection record data
	 - for sample points not co-located with intersections, derive estimates by:
		 - accounting for distances
		 - calculating proximity-weighted average of density statistics for each sample point


In [12]:
#Cumulative opportunities (binary)
#1 if d <= access_dist
#0 if d > access_dist
def binary_access_score(df, distance_names, threshold=500):
    """
    Calculate accessibiity score using binary measure: 1 if access <= access_dist, 0 otherwise

    Parameters
    ----------
    df: DataFrame
        DataFrame with origin-destination distances
    distance_names: list
        list of original distance field names
    threshold: int
        access distance threshold, default is 500 meters

    Returns
    -------
    DataFrame
    """
    df1 = (df[distance_names] <= threshold).fillna(0).astype(int)
    return df1


#Soft threshold access score
#Higgs, C., Badland, H., Simons, K. et al. (2019) The Urban Liveability Index
def soft_access_score(df, distance_names, threshold=500, k=5):
    """
    Calculate accessibiity score using soft threshold approach:
    1 / (1+ e ^(k *((dist-access_dist)/access_dist)))
    Parameters
    ----------
    df: DataFrame
        DataFrame with origin-destination distances
    distance_names: list
        list of original distance field names
    threshold: int
        access distance threshold, default is 500 meters
    k: int
        the slope of decay, default is 5
    Returns
    -------
    DataFrame
    """
    df1 = (1 / (1+numpy.exp(k * ((df[distance_names]-threshold) / threshold)))).fillna(0).astype(float)
    return df1

#Cumulative-Gaussian
#Reference: Vale, D. S., & Pereira, M. (2017).
#The influence of the impedance function on gravity-based pedestrian accessibility measures
def Cumulative_Gaussian_access_score(df, distance_names, threshold=500, k=129842):
    """
    Calculate accessibiity score using Cumulative-Gaussian approach:
    1 if d <= access_dist ; otherwise, e ^(-1 *((d^2)/k)) if d > access_dist
    Parameters
    ----------
    df: DataFrame
        DataFrame with origin-destination distances
    distance_names: list
        list of field names for distance records
    threshold: int
        access distance threshold
    k: int
        the slope of decay
    Returns
    -------
    DataFrame
    """
    df1 = df[distance_names].copy()
    df1 = df1.astype(float)
    df1[df1<=threshold] = 1
    df1[df1>threshold] = numpy.exp(-1 * (((df1[df1>threshold]-threshold)**2) / k))
    df1 = df1.fillna(0).astype(float)
    return(df1)


In [13]:
#binary
binary_names = [f"{x.replace('nearest_node','access')}_binary" for x in distance_names]
samplePointsData[binary_names] = binary_access_score(samplePointsData, distance_names, parameters['accessibility_distance'])
samplePointsData[binary_names]

#soft
soft_names = [f"{x.replace('nearest_node','access')}_soft" for x in distance_names]
samplePointsData[soft_names] = soft_access_score(samplePointsData, distance_names, parameters['accessibility_distance'])
samplePointsData[soft_names]

#c-m
c_m_names = [f"{x.replace('nearest_node','access')}_score" for x in distance_names]
samplePointsData[c_m_names] = Cumulative_Gaussian_access_score(samplePointsData, distance_names, parameters['accessibility_distance'])
samplePointsData[c_m_names]


,sp_access_fresh_food_market_score,sp_access_convenience_score,sp_access_pt_osm_any_score,sp_access_public_open_space_any_score,sp_access_public_open_space_large_score,sp_access_pt_gtfs_any_score,sp_access_pt_gtfs_freq_30_score,sp_access_pt_gtfs_freq_20_score
point_id,,,,,,,,
1,1.0,1.000000,1.0,1.0,1.0,0.0,0.0,0.0
2,1.0,0.000000,1.0,1.0,1.0,0.0,0.0,0.0
7,1.0,1.000000,1.0,1.0,1.0,0.0,0.0,0.0
8,1.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0
9,1.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
26336,1.0,1.000000,1.0,1.0,1.0,0.0,0.0,0.0
26337,1.0,1.000000,1.0,1.0,1.0,0.0,0.0,0.0
26338,1.0,1.000000,1.0,1.0,1.0,0.0,0.0,0.0


In [71]:
print("Calculating sample point specific analyses ...")
# Defined in generated config file, e.g. daily living score, walkability index, etc
for analysis in config['sample_point_analyses']:
    print(f"\t - {analysis}")
    for var in config['sample_point_analyses'][analysis]:
        columns = config['sample_point_analyses'][analysis][var]['columns']
        formula = config['sample_point_analyses'][analysis][var]['formula']
        axis    = config['sample_point_analyses'][analysis][var]['axis']
        if formula == "sum":
            samplePointsData[var] = samplePointsData[columns].sum(axis=axis)
        if formula == "max":
            samplePointsData[var] = samplePointsData[columns].max(axis=axis)
        if formula == "sum_of_z_scores":
            samplePointsData[var] =  ((samplePointsData[columns] -  samplePointsData[columns].mean()) \
                                             / samplePointsData[columns].std()).sum(axis=1)       

# hex_id and edge_ogc_fid are integers
samplePointsData[samplePointsData.columns[0:2]] = samplePointsData[samplePointsData.columns[0:2]].astype(int)
# remaining non-geometry fields are float
samplePointsData[samplePointsData.columns[3:]] = samplePointsData[samplePointsData.columns[3:]].astype(float)

Calculating sample point specific analyses ...
	 - Best PT (any) access score
	 - Daily living score
	 - Walkability index


In [72]:
len(samplePointsData)

11499

In [73]:
samplePointsData.columns

Index(['hex_id', 'edge_ogc_fid', 'geometry',
       'sp_nearest_node_fresh_food_market', 'sp_nearest_node_convenience',
       'sp_nearest_node_pt_osm_any', 'sp_nearest_node_public_open_space_any',
       'sp_nearest_node_public_open_space_large',
       'sp_nearest_node_pt_gtfs_any', 'sp_nearest_node_pt_gtfs_freq_30',
       'sp_nearest_node_pt_gtfs_freq_20', 'sp_local_nh_avg_pop_density',
       'sp_local_nh_avg_intersection_density',
       'sp_access_fresh_food_market_binary', 'sp_access_convenience_binary',
       'sp_access_pt_osm_any_binary', 'sp_access_public_open_space_any_binary',
       'sp_access_public_open_space_large_binary',
       'sp_access_pt_gtfs_any_binary', 'sp_access_pt_gtfs_freq_30_binary',
       'sp_access_pt_gtfs_freq_20_binary', 'sp_access_pt_any_binary',
       'sp_daily_living_score', 'sp_walkability_index'],
      dtype='object')

In [77]:
samplePointsData['sp_daily_living_score']

point_id
1        2.866221
2        1.548086
7        2.419968
8        2.632533
9        2.562366
           ...   
26336    2.295686
26337    2.497561
26338    2.376695
26339    1.620893
26340    2.181547
Name: sp_daily_living_score, Length: 11499, dtype: float64

In [ ]:
col='sp_walkability_index'

fig, ax = plt.subplots(figsize=(5, 5))

#plot indicators
ax = samplePointsData.plot(ax=ax, column=col, scheme='NaturalBreaks', k=6, cmap='inferno_r', edgecolor='none')

ax.set_title(config['study_region'], fontsize=9)
ax.set_axis_off()

# add a title to the figure
fig.suptitle('Local Sample Point walkability', y=0.95, fontsize=10, weight='bold')